In [ ]:
# Install lightgbm if not already installed
%pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.6 MB ? eta -:--:--Downloading lightgbm-4.6.0-py3-none-manylinux_2_28_x86_64.whl (3.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.5 MB/s eta 0:00:00a 0:00:01


# Import the libraries

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
print("added")

added


### Loading of the train data

In [8]:
# Load the dataset
print("Loading data...")
try:
    train_orders = pd.read_csv('Train/orders.csv', low_memory=False)
    train_customers = pd.read_csv('Train/train_customers.csv')
    train_locations = pd.read_csv('Train/train_locations.csv')
    vendors = pd.read_csv('Train/vendors.csv')
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure all CSV files are in the same directory as the script.")
    exit()

print("Data loaded successfully.")

Loading data...
Data loaded successfully.
Data loaded successfully.


In [ ]:
print("merging data... ")
print("train_orders columns:", train_orders.columns.tolist())
print("train_customers columns:", train_customers.columns.tolist())
print("vendors columns:", vendors.columns.tolist())
print("train_locations columns:", train_locations.columns.tolist())

# Check for correct column name in train_orders and train_locations for location number
orders_location_col = None
locations_location_col = None

for col in ['location_number', 'LOCATION_NUMBER']:
    if col in train_orders.columns:
        orders_location_col = col
    if col in train_locations.columns:
        locations_location_col = col

if orders_location_col is None or locations_location_col is None:
    raise KeyError("Could not find a matching location number column in both train_orders and train_locations.")

train_merged = train_orders.merge(train_customers, on='customer_id', how='left')
train_merged = train_merged.merge(vendors, left_on='vendor_id', right_on='id', how='left')
train_merged = train_merged.merge(
    train_locations,
    left_on=['customer_id', orders_location_col],
    right_on=['customer_id', locations_location_col],
    how='left'
)
print("Data merged successfully.")
train_merged.to_csv('Train/train_merged.csv', index=False)
print("Merged data saved to Train/train_merged.csv")

merging data... 
train_orders columns: ['order_id', 'customer_id', 'item_count', 'grand_total', 'payment_mode', 'promo_code', 'vendor_discount_amount', 'promo_code_discount_percentage', 'is_favorite', 'is_rated', 'vendor_rating', 'driver_rating', 'deliverydistance', 'preparationtime', 'delivery_time', 'order_accepted_time', 'driver_accepted_time', 'ready_for_pickup_time', 'picked_up_time', 'delivered_time', 'delivery_date', 'vendor_id', 'created_at', 'LOCATION_NUMBER', 'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR']
train_customers columns: ['customer_id', 'gender', 'dob', 'status', 'verified', 'language', 'created_at', 'updated_at']
vendors columns: ['id', 'authentication_id', 'latitude', 'longitude', 'vendor_category_en', 'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2', 'prepration_time', 'commission', 'is_haked_delivering', 'discount_percentage', 'status', 'verified', 'rank', 'language', 'vendor_rating', 'sunday_from_time1', 'sunday_t

KeyError: 'location_number'